In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier

In [13]:
result = pd.DataFrame({'uuid': X_test.index, 'pd': pipe.predict_proba(X_test)[:, 1]})
result.head()

,uuid,pd
0,20daeb41-1bf5-4c0d-922f-4a09745721b1,0.016774
1,1eb41ded-2b7d-4b90-a44c-15cec7a5df76,0.001879
2,8e3f9c46-9ff4-4519-aa8b-668ec73c9e9f,0.005556
3,00356e40-17de-454c-80ad-e996d9b90744,0.001247
4,251f3906-ac22-43c9-b87b-3b141398c5f1,0.008726


In [14]:
raw_data = pd.read_csv('dataset.csv', sep=';', index_col='uuid')

In [15]:
raw_data.columns

Index(['default', 'account_amount_added_12_24m', 'account_days_in_dc_12_24m',
       'account_days_in_rem_12_24m', 'account_days_in_term_12_24m',
       'account_incoming_debt_vs_paid_0_24m', 'account_status',
       'account_worst_status_0_3m', 'account_worst_status_12_24m',
       'account_worst_status_3_6m', 'account_worst_status_6_12m', 'age',
       'avg_payment_span_0_12m', 'avg_payment_span_0_3m', 'merchant_category',
       'merchant_group', 'has_paid', 'max_paid_inv_0_12m',
       'max_paid_inv_0_24m', 'name_in_email',
       'num_active_div_by_paid_inv_0_12m', 'num_active_inv',
       'num_arch_dc_0_12m', 'num_arch_dc_12_24m', 'num_arch_ok_0_12m',
       'num_arch_ok_12_24m', 'num_arch_rem_0_12m',
       'num_arch_written_off_0_12m', 'num_arch_written_off_12_24m',
       'num_unpaid_bills', 'status_last_archived_0_24m',
       'status_2nd_last_archived_0_24m', 'status_3rd_last_archived_0_24m',
       'status_max_archived_0_6_months', 'status_max_archived_0_12_months',
       

In [16]:
y = raw_data['default']
X = raw_data.drop(columns='default')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [23]:
class CategoricalEncoder(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

    def transform(self, X, y=None):
        assert isinstance(X, pd.DataFrame)
        X = X.dropna(axis=1)
        X_hot = pd.get_dummies(X, drop_first=True)
        return X_hot

    def fit(self, X, y=None):
        return self
    
from sklearn.preprocessing import StandardScaler

class MyScaler(BaseEstimator):
    
    def __init__(self):
        self.scaler = StandardScaler()
    
    def fit(self, X, y=None):
        assert isinstance(X, pd.DataFrame)
        X = X.dropna(axis=1)
        self.scaler.fit(X,y)
        return self
    
    def transform(self, X, y=None):
        self.scaler.transform(X,y)
        return X, y

#r_scaler = StandardScaler()
#for col in X.columns:
#    r_scaler.fit(X[[col]])
#    X[col] = r_scaler.transform(X[[col]])

In [31]:
from sklearn.compose import ColumnTransformer

cols = X_train.columns

cat_cols = ['merchant_category',
       'merchant_group', 'name_in_email']

num_cols = ['account_amount_added_12_24m', 'account_days_in_dc_12_24m',
       'account_days_in_rem_12_24m', 'account_days_in_term_12_24m',
       'account_incoming_debt_vs_paid_0_24m', 'account_status',
       'account_worst_status_0_3m', 'account_worst_status_12_24m',
       'account_worst_status_3_6m', 'account_worst_status_6_12m', 'age',
       'avg_payment_span_0_12m', 'avg_payment_span_0_3m', 'has_paid', 'max_paid_inv_0_12m',
       'max_paid_inv_0_24m',
       'num_active_div_by_paid_inv_0_12m', 'num_active_inv',
       'num_arch_dc_0_12m', 'num_arch_dc_12_24m', 'num_arch_ok_0_12m',
       'num_arch_ok_12_24m', 'num_arch_rem_0_12m',
       'num_arch_written_off_0_12m', 'num_arch_written_off_12_24m',
       'num_unpaid_bills', 'status_last_archived_0_24m',
       'status_2nd_last_archived_0_24m', 'status_3rd_last_archived_0_24m',
       'status_max_archived_0_6_months', 'status_max_archived_0_12_months',
       'status_max_archived_0_24_months', 'recovery_debt',
       'sum_capital_paid_account_0_12m', 'sum_capital_paid_account_12_24m',
       'sum_paid_inv_0_12m', 'time_hours', 'worst_status_active_inv']

#transformer = ColumnTransformer(transformers, *, remainder='drop')
encoder = ColumnTransformer([('categorical', CategoricalEncoder(), cols)]) #,
#                            ('numerical', MyScaler(), num_cols)])

In [19]:
def get_data():
    raw_data = '../KlarnaModel/data/dataset.csv'
    df = pd.read_csv(raw_data, sep=';', index_col='uuid')
    predict = df[df['default'].isnull()].copy()
    df.drop(index=predict.index, inplace=True)
    return df, predict 

def clean_data(df):
    df.dropna(axis=1, inplace=True)
    df.drop_duplicates(inplace=True)
    y = df['default']
    df.drop(columns='default', inplace=True)
    return df, y

def split_data(df, y):
    X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.3, random_state=42)
    return X_train, X_test, y_train, y_test

def get_pipeline(X):
    cols = X.columns
    encoder = ColumnTransformer([('categorical', OneHotEncoder(handle_unknown='ignore'), ['merchant_category', 'merchant_group', 'name_in_email'])], remainder='passthrough')
    pipe = Pipeline(steps=[
        ('encoder', encoder),
        ('classifier', RandomForestClassifier(n_estimators=10, max_depth=8, max_features=20))
    ])
    return pipe

def train(X_train, y_train, pipeline):
    return pipeline.fit(X_train, y_train)

def evaluate(X, y, pipeline):
    return pipeline.score(X,y)

In [20]:
df, pred = get_data()

In [21]:
df, y = clean_data(df)

In [22]:
df.head()

,account_amount_added_12_24m,age,merchant_category,merchant_group,has_paid,max_paid_inv_0_12m,max_paid_inv_0_24m,name_in_email,num_active_inv,num_arch_dc_0_12m,...,status_2nd_last_archived_0_24m,status_3rd_last_archived_0_24m,status_max_archived_0_6_months,status_max_archived_0_12_months,status_max_archived_0_24_months,recovery_debt,sum_capital_paid_account_0_12m,sum_capital_paid_account_12_24m,sum_paid_inv_0_12m,time_hours
uuid,,,,,,,,,,,,,,,,,,,,,
63f69b2c-8b1c-4740-b78d-52ed9a4515ac,0,20,Dietary supplements,Health & Beauty,True,31638.0,31638.0,no_match,2,0,...,1,1,1,1,1,0,0,0,178839,9.653333
0e961183-8c15-4470-9a5e-07a1bd207661,0,50,Books & Magazines,Entertainment,True,13749.0,13749.0,F+L,0,0,...,1,1,1,2,2,0,0,0,49014,13.181389
d8edaae6-4368-44e0-941e-8328f203e64e,0,22,Diversified entertainment,Entertainment,True,29890.0,29890.0,L1+F,1,0,...,1,1,1,2,2,0,0,0,124839,11.561944
0095dfb6-a886-4e2a-b056-15ef45fdb0ef,0,36,Diversified entertainment,Entertainment,True,40040.0,40040.0,F1+L,1,0,...,1,1,1,1,1,0,0,0,324676,15.751111
c8f8b835-5647-4506-bf15-49105d8af30b,0,25,Electronic equipment & Related accessories,Electronics,True,7100.0,7100.0,F+L,0,0,...,0,0,1,1,1,0,0,0,7100,12.698611


In [10]:
df.describe()

,account_amount_added_12_24m,age,max_paid_inv_0_12m,max_paid_inv_0_24m,num_active_inv,num_arch_dc_0_12m,num_arch_dc_12_24m,num_arch_ok_0_12m,num_arch_ok_12_24m,num_arch_rem_0_12m,...,status_2nd_last_archived_0_24m,status_3rd_last_archived_0_24m,status_max_archived_0_6_months,status_max_archived_0_12_months,status_max_archived_0_24_months,recovery_debt,sum_capital_paid_account_0_12m,sum_capital_paid_account_12_24m,sum_paid_inv_0_12m,time_hours
count,8.997400e+04,89974.000000,89974.000000,89974.000000,89974.000000,89974.000000,89974.000000,89974.000000,89974.000000,89974.000000,...,89974.000000,89974.000000,89974.000000,89974.000000,89974.00000,89974.000000,89974.000000,89974.000000,8.997400e+04,89974.000000
mean,1.227643e+04,36.019206,9224.295697,11244.566819,0.598717,0.061762,0.059751,7.289939,6.380843,0.467268,...,0.828006,0.744515,0.800276,1.051570,1.22581,4.044491,10833.900649,6580.000578,3.921437e+04,15.336407
std,3.546390e+04,12.999712,13577.028797,15257.624405,1.544356,0.373624,0.368216,16.074805,15.398063,1.351363,...,0.622770,0.634201,0.719560,0.785786,0.83364,167.326016,26494.323856,19192.973589,8.940091e+04,5.030695
min,0.000000e+00,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000e+00,0.000278
25%,0.000000e+00,25.000000,2000.000000,3340.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,1.00000,0.000000,0.000000,0.000000,2.630000e+03,11.629167
50%,0.000000e+00,34.000000,6057.000000,7585.000000,0.000000,0.000000,0.000000,2.000000,2.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.00000,0.000000,0.000000,0.000000,1.600000e+04,15.800833
75%,4.984750e+03,45.000000,11380.000000,13812.000000,1.000000,0.000000,0.000000,7.000000,6.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,2.00000,0.000000,9098.750000,100.000000,4.389975e+04,19.548056
max,1.128775e+06,100.000000,279000.000000,279000.000000,47.000000,16.000000,13.000000,261.000000,313.000000,42.000000,...,5.000000,5.000000,3.000000,5.000000,5.00000,36479.000000,571475.000000,341859.000000,2.962870e+06,23.999722


In [61]:
df[df['age']==100]

,account_amount_added_12_24m,age,merchant_category,merchant_group,has_paid,max_paid_inv_0_12m,max_paid_inv_0_24m,name_in_email,num_active_inv,num_arch_dc_0_12m,...,status_2nd_last_archived_0_24m,status_3rd_last_archived_0_24m,status_max_archived_0_6_months,status_max_archived_0_12_months,status_max_archived_0_24_months,recovery_debt,sum_capital_paid_account_0_12m,sum_capital_paid_account_12_24m,sum_paid_inv_0_12m,time_hours
uuid,,,,,,,,,,,,,,,,,,,,,
839d4a5c-ae07-4c7b-9a9d-bee8161fb8dc,0,100,Personal care & Body improvement,Health & Beauty,False,0.0,0.0,Nick,1,0,...,0,0,0,0,0,0,0,0,0,16.975556


In [23]:
X_train, X_test, y_train, y_test = split_data(df, y)

In [24]:
pipe = get_pipeline(X_train)

In [25]:
train(X_train, y_train, pipe)

Pipeline(memory=None,
     steps=[('encoder', ColumnTransformer(n_jobs=None, remainder='passthrough', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('categorical', OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='ignore',
       n...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [26]:
evaluate(X_test, y_test, pipe)

0.98558885637017

In [27]:
len(X_test.columns)

26

In [28]:
def predict_prob(X, pipe):
    result = pd.DataFrame({'uuid': X.index, 'pd': pipe.predict_proba(X)[:, 1]})
    return result

In [31]:
pred.head()

,account_amount_added_12_24m,age,merchant_category,merchant_group,has_paid,max_paid_inv_0_12m,max_paid_inv_0_24m,name_in_email,num_active_inv,num_arch_dc_0_12m,...,status_2nd_last_archived_0_24m,status_3rd_last_archived_0_24m,status_max_archived_0_6_months,status_max_archived_0_12_months,status_max_archived_0_24_months,recovery_debt,sum_capital_paid_account_0_12m,sum_capital_paid_account_12_24m,sum_paid_inv_0_12m,time_hours
uuid,,,,,,,,,,,,,,,,,,,,,
6f6e6c6a-2081-4e6b-8eb3-4fd89b54b2d7,0,20,Youthful Shoes & Clothing,Clothing & Shoes,True,7225.0,7225.0,F,0,0,...,1,1,1,1,1,0,8815,0,27157,19.895556
f6f6d9f3-ef2b-4329-a388-c6a687f27e70,0,64,Personal care & Body improvement,Health & Beauty,False,0.0,0.0,L1+F,0,0,...,0,0,0,0,0,0,0,0,0,0.236667
e9c39869-1bc5-4375-b627-a2df70b445ea,50956,28,Diversified entertainment,Entertainment,True,91980.0,91980.0,F,0,0,...,1,2,1,1,3,0,36163,39846,93760,20.332778
6beb88a3-9641-4381-beb6-c9a208664dd0,35054,31,Concept stores & Miscellaneous,"Leisure, Sport & Hobby",True,1790.0,1790.0,F+L,0,0,...,0,0,2,2,2,0,62585,0,1790,6.201111
bb89b735-72fe-42a4-ba06-d63be0f4ca36,0,30,Diversified electronics,Electronics,True,0.0,0.0,F1+L,0,0,...,0,0,0,0,0,0,14295,0,0,8.451111


In [32]:
predict_prob(pred, pipe)

,uuid,pd
0,6f6e6c6a-2081-4e6b-8eb3-4fd89b54b2d7,0.003805
1,f6f6d9f3-ef2b-4329-a388-c6a687f27e70,0.020039
2,e9c39869-1bc5-4375-b627-a2df70b445ea,0.026714
3,6beb88a3-9641-4381-beb6-c9a208664dd0,0.019512
4,bb89b735-72fe-42a4-ba06-d63be0f4ca36,0.025148
5,e4eede99-76a3-4437-a540-3059a1eff67c,0.007726
6,a2af8d9e-9f81-4185-8fff-b2ec49d681a6,0.001121
7,ec910486-1e66-402a-80f2-08c6f04a9a1b,0.033790
8,08973cf0-646a-4fa7-9f1f-d03f76ffd59c,0.003027
9,0591fb4e-5b48-4bac-bce7-f2d5d141e976,0.001204


In [33]:
pred.to_csv('../KlarnaModel/data/test_app.csv')

In [34]:
raw_data = '../KlarnaModel/data/dataset.csv'
df = pd.read_csv(raw_data, sep=';', index_col='uuid')
predict = df[df['default'].isnull()].copy()
df.drop(index=predict.index, inplace=True)
df.to_csv('../KlarnaModel/data/klarna_df.csv')

In [2]:
import pandas as pd

predict = pd.read_csv('../KlarnaModel/data/data_to_predict.csv')

In [3]:
import joblib

pipe = joblib.load('../model/my_pipeline.joblib')

In [4]:
C = ['account_amount_added_12_24m', 'account_days_in_dc_12_24m',
       'account_days_in_rem_12_24m', 'account_days_in_term_12_24m',
       'account_incoming_debt_vs_paid_0_24m', 'account_status',
       'account_worst_status_0_3m', 'account_worst_status_12_24m',
       'account_worst_status_3_6m', 'account_worst_status_6_12m', 'age',
       'avg_payment_span_0_12m', 'avg_payment_span_0_3m', 'merchant_category',
       'merchant_group', 'has_paid', 'max_paid_inv_0_12m',
       'max_paid_inv_0_24m', 'name_in_email',
       'num_active_div_by_paid_inv_0_12m', 'num_active_inv',
       'num_arch_dc_0_12m', 'num_arch_dc_12_24m', 'num_arch_ok_0_12m',
       'num_arch_ok_12_24m', 'num_arch_rem_0_12m',
       'num_arch_written_off_0_12m', 'num_arch_written_off_12_24m',
       'num_unpaid_bills', 'status_last_archived_0_24m',
       'status_2nd_last_archived_0_24m', 'status_3rd_last_archived_0_24m',
       'status_max_archived_0_6_months', 'status_max_archived_0_12_months',
       'status_max_archived_0_24_months', 'recovery_debt',
       'sum_capital_paid_account_0_12m', 'sum_capital_paid_account_12_24m',
       'sum_paid_inv_0_12m', 'time_hours', 'worst_status_active_inv']

def predict_result(X, pipe):
    if 'uuid' in X.columns:
        result = X[['uuid']].copy()
    else:
        result = pd.DataFrame(index=X.index, columns=['pd'])
    X_ =pd.DataFrame(X, columns=C)
    result['pd'] = pipe.predict_proba(X_)[:, 1]
    print(result)
    return result

In [5]:
result = predict_result(predict, pipe)

                                      uuid            pd
0     6f6e6c6a-2081-4e6b-8eb3-4fd89b54b2d7  1.781153e-08
1     f6f6d9f3-ef2b-4329-a388-c6a687f27e70  3.906438e-06
2     e9c39869-1bc5-4375-b627-a2df70b445ea  2.309321e-03
3     6beb88a3-9641-4381-beb6-c9a208664dd0  1.816762e-02
4     bb89b735-72fe-42a4-ba06-d63be0f4ca36  1.054551e-06
5     e4eede99-76a3-4437-a540-3059a1eff67c  1.436959e-09
6     a2af8d9e-9f81-4185-8fff-b2ec49d681a6  7.361703e-09
7     ec910486-1e66-402a-80f2-08c6f04a9a1b  8.904816e-03
8     08973cf0-646a-4fa7-9f1f-d03f76ffd59c  1.520295e-09
9     0591fb4e-5b48-4bac-bce7-f2d5d141e976  9.076759e-11
10    0d08fe69-ca57-433d-a6db-83970f6d11d8  2.511096e-07
11    53cfc224-c727-467c-b11b-585ce4137d83  7.823398e-12
12    7d7c6bed-ce10-45c3-8ef4-55e84ae7f9d9  7.365976e-08
13    d1087a43-7eb0-47ad-839c-14a0e8c5c0da  4.225152e-15
14    5c678d63-1da0-42e8-aed0-bafda88ffe98  1.114087e-09
15    8d1bd7fd-d065-425f-a35f-044a7af0217d  1.218567e-06
16    a0e7a004-a491-491b-a95c-d

In [6]:
result.to_csv('../KlarnaModel/data/result.csv')

In [2]:
import joblib

model = joblib.load('xgboost_best_model.joblib')

In [3]:
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9232364924465264, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.5490797147630709, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=776, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=0.3337247329616153, tree_method='exact',
              validate_parameters=1, verbosity=None)